In [2]:
import pandas as pd
import os

In [3]:
# Đọc các file CSV
df1 = pd.read_csv(os.path.join('../output/traffic_images_dataset_v1.csv'))
df2 = pd.read_csv(os.path.join('../output/traffic_images_dataset_v2.csv'))

# Merge 2 dataframe
merged_df = pd.concat([df1, df2], ignore_index=True)

# Hiển thị kích thước của DataFrame
print(f"Kích thước của DataFrame sau khi merge:")
print(f"Số lượng dòng: {merged_df.shape[0]}")
print(f"Số lượng cột: {merged_df.shape[1]}")

# Hiển thị tên các cột
print("\nDanh sách các cột:")
print(merged_df.columns.tolist())

Kích thước của DataFrame sau khi merge:
Số lượng dòng: 19891
Số lượng cột: 8

Danh sách các cột:
['title', 'original_url', 'thumbnail_url', 'source_website', 'resolution', 'search_query', 'page_number', 'local_path']


In [4]:
# Kiểm tra số lượng URL trùng lặp
duplicate_urls = merged_df[merged_df.duplicated(subset=['original_url'], keep='first')]
print(f"\nSố lượng URL trùng lặp: {len(duplicate_urls)}")

# Kiểm tra số lượng giá trị null trong từng cột
print("\nSố lượng giá trị null trong mỗi cột:")
print(merged_df.isnull().sum())

# Lọc bỏ các dòng trùng lặp (giữ lại dòng đầu tiên)
merged_df_no_duplicates = merged_df.drop_duplicates(subset=['original_url'], keep='first')

# Lọc bỏ các dòng có giá trị null
merged_df_clean = merged_df_no_duplicates.dropna()

# Hiển thị kích thước của DataFrame sau khi lọc
print(f"\nKích thước DataFrame sau khi lọc:")
print(f"Ban đầu: {merged_df.shape}")
print(f"Sau khi lọc trùng lặp: {merged_df_no_duplicates.shape}")
print(f"Sau khi lọc null: {merged_df_clean.shape}")


Số lượng URL trùng lặp: 7404

Số lượng giá trị null trong mỗi cột:
title                0
original_url         0
thumbnail_url        0
source_website       0
resolution           0
search_query         0
page_number          0
local_path        3119
dtype: int64

Kích thước DataFrame sau khi lọc:
Ban đầu: (19891, 8)
Sau khi lọc trùng lặp: (12487, 8)
Sau khi lọc null: (10840, 8)


In [7]:
# Chọn các cột cần thiết
selected_columns = ['original_url', 'source_website', 'resolution', 'search_query', 'local_path']
final_df = merged_df_clean[selected_columns].copy()

# Thêm cột short_caption mới (khởi tạo với giá trị rỗng)
final_df['short_caption'] = ''

# Hiển thị thông tin về DataFrame sau khi xử lý
print("Danh sách các cột sau khi xử lý:")
print(final_df.columns.tolist())
print(f"\nKích thước DataFrame cuối cùng: {final_df.shape}")

# Hiển thị 5 dòng đầu tiên để kiểm tra
print("\n5 dòng đầu tiên của DataFrame:")
final_df.head()

Danh sách các cột sau khi xử lý:
['original_url', 'source_website', 'resolution', 'search_query', 'local_path', 'short_caption']

Kích thước DataFrame cuối cùng: (10840, 6)

5 dòng đầu tiên của DataFrame:


,original_url,source_website,resolution,search_query,local_path,short_caption
0,https://cand.com.vn/Files/Image/chienthang/202...,Công an Nhân dân,600x400,vỉa hè đường phố việt nam,images/vỉa hè đường phố việt nam_1736910371.jpg,
1,https://cafebiz.cafebizcdn.vn/1621233102540021...,CafeBiz,2000x1500,vỉa hè đường phố việt nam,images/vỉa hè đường phố việt nam_1736910372.jpg,
2,https://cdn.tiasang.com.vn/tiasang-media/2023/...,Tạp chí Tia sáng,1901x1271,vỉa hè đường phố việt nam,images/vỉa hè đường phố việt nam_1736910372.jpg,
3,https://cand.com.vn/Files/Image/chienthang/202...,Công an Nhân dân,660x451,vỉa hè đường phố việt nam,images/vỉa hè đường phố việt nam_1736910372.jpg,
4,https://icdn.dantri.com.vn/thumb_w/680/dansinh...,Dân sinh - Dân trí,680x454,vỉa hè đường phố việt nam,images/vỉa hè đường phố việt nam_1736910372.jpg,


In [ ]:
import requests
from tqdm import tqdm
import urllib3
import time  
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def check_image_url(url, max_retries=2):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    for attempt in range(max_retries):
        try:
            # Chỉ gửi HEAD request để kiểm tra metadata, không tải nội dung
            response = requests.head(
                url,
                headers=headers,
                timeout=5,  # Giảm timeout xuống vì chỉ check metadata
                verify=False,
                allow_redirects=True
            )
            
            if response.status_code == 200:
                content_type = response.headers.get('content-type', '')
                if 'image' in content_type:
                    return True, None
                else:
                    return False, "Không phải file ảnh"
            else:
                return False, f"Lỗi HTTP {response.status_code}"
                
        except requests.exceptions.SSLError:
            if attempt == max_retries - 1:
                return False, "Lỗi SSL"
        except requests.exceptions.Timeout:
            if attempt == max_retries - 1:
                return False, "Timeout"
        except requests.exceptions.ConnectionError:
            if attempt == max_retries - 1:
                return False, "Lỗi kết nối"
            time.sleep(1)
        except Exception as e:
            return False, f"Lỗi không xác định: {str(e)}"
    
    return False, f"Thất bại sau {max_retries} lần thử"

# Kiểm tra toàn bộ dataset
error_stats = {}
failed_urls = []
success_count = 0

# Kiểm tra URLs
for idx, row in tqdm(final_df.iterrows(), total=len(final_df), desc="Đang kiểm tra URLs"):
    url = row['original_url']
    success, error_message = check_image_url(url)
    
    if success:
        success_count += 1
    else:
        failed_urls.append((url, error_message))
        error_stats[error_message] = error_stats.get(error_message, 0) + 1
    
    # Thêm delay nhỏ để tránh quá tải server
    time.sleep(0.1)

# Lọc bỏ các URL thất bại
failed_url_list = [url for url, _ in failed_urls]
clean_df = final_df[~final_df['original_url'].isin(failed_url_list)]

# Thống kê kết quả
print("\n=== THỐNG KÊ KẾT QUẢ KIỂM TRA URL ===")
print(f"Tổng số URL: {len(final_df)}")
print(f"Số URL hợp lệ: {success_count}")
print(f"Số URL không hợp lệ: {len(failed_urls)}")

print("\n=== CHI TIẾT LỖI ===")
for error_type, count in error_stats.items():
    print(f"{error_type}: {count} URLs")

print("\n=== THÔNG TIN DATASET SAU KHI LỌC ===")
print(f"Kích thước ban đầu: {final_df.shape}")
print(f"Kích thước sau khi lọc: {clean_df.shape}")
print("\nPhân bố dữ liệu theo source_website:")
print(clean_df['source_website'].value_counts().head())

# Lưu danh sách URL thất bại
with open('invalid_urls.txt', 'w', encoding='utf-8') as f:
    f.write("URL,Lỗi\n")
    for url, error in failed_urls:
        f.write(f"{url},{error}\n")

# Lưu DataFrame đã được lọc
clean_df.to_csv('./csv/valid_urls_dataset_v12.csv', index=False, encoding="utf-8-sig")